In [149]:
import time
import pygame
import pygame.midi
import math
import numpy
from json import load

import tensorflow as tf
from keras.models import load_model

from music21 import *

################################################################################################################
duration_seq = [] 

In [150]:
def print_midi_info():
    pygame.midi.init()
    for i in range(pygame.midi.get_count()):
        print(pygame.midi.get_device_info(i))
    
    print("Default in: ", pygame.midi.get_default_input_id())
    print("Default out: ", pygame.midi.get_default_output_id())
    pygame.midi.quit()

print_midi_info()

(b'MMSystem', b'Microsoft MIDI Mapper', 0, 1, 0)
(b'MMSystem', b'Microsoft GS Wavetable Synth', 0, 1, 0)
Default in:  -1
Default out:  0


In [151]:
notes = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']

def num_to_note(num):
    note = notes[num % 12]
    note = note + str(math.ceil(num/12)-1)
    return note


In [152]:
def take_input(in_id = None, out_id = None):
    pygame.midi.init()

    if not in_id:
        in_id = pygame.midi.get_default_input_id()

    if not out_id:
        out_id = pygame.midi.get_default_output_id()

    i = pygame.midi.Input(in_id)
    o = pygame.midi.Output(out_id,latency=0,buffer_size=1)
    o.set_instrument(0)

    pygame.display.set_mode((1,1))
    note_count = 0

    #Turn these arrays into nupy.array()s like this: final_list = np.array(list(zip(note_seq,dur_seq)))
                # Will need to map ints from keyboard to chars representing the note (ie 'C3' 'A-4' etc) 
                # then return it and in produce_note_model create a new function and map the chars to their 
                # respective ints before sending them to produce_song 

                # have to caulculate the differnece in the start and end of notes, round the result to the 
                # nearest musical "pause" then send the word along with the string for note (bpm 120)
            #Chords?
                # PLeASe nO
        #I'll need to store the notes that have been pressed to check timestamp for release of the note


        #A mid C is 60, so using mod 12 to get notes and octave is important.

    #Format: [[pitch, start, stop], [pitch,start,stop], ...]
    note_seq = [] 
    while note_count < 10:
        if i.poll(): #if a note has been presseed
            midi_event = i.read(1) #grab note played

            if midi_event[0][0][0] == 144: #if note on
                print(midi_event)
                print(num_to_note(midi_event[0][0][1]))
                note = midi_event[0][0][1]
                time_stamp = midi_event[0][1] # in milliseconds I think

                note_seq.append([note, time_stamp, 0])

                o.note_on(note, 120)               

            if midi_event[0][0][0] == 128: #if note off
                print(midi_event)
                note = midi_event[0][0][1]
                time_stamp = midi_event[0][1]
                for n in range(len(note_seq)-1, -1, -1):
                    if(note_seq[n][0] == note and note_seq[n][2] == 0):
                        note_seq[n][2] = time_stamp
                o.note_off(note, 120)
               
                note_count += 1  #DOES NOT ACCOUNT FOR CHORDS

    ################################################################################################################
    for i in note_seq:
        duration_seq.append(note_seq[i][2]-note_seq[i,1])

            #o.write_short(midi_event[0][0][0], midi_event[0][0][1], 120) #write start/stop, note, velocity to output
    print(note_seq)
    time.sleep(1) #Give last note time to play
    i.close()
    o.close()
    pygame.midi.quit()
    pygame.quit()

#take_input(out_id=4)



def read_model():
    existent_model = load_model('models/best_model_note.h5')
    existent_model.summary()
    

read_model()
int_to_note = loads("int_to_note.json")
output_notes = []
for i in duration_seq:
    new_note = note.Note(int_to_note[i]) 
    new_note.offset = i
    new_note.duration = duration_seq
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8, 1)]            0         
                                                                 
 dense (Dense)               (None, 8, 128)            256       
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 40)                5160      
                                                                 
Total params: 153,512
Trainable params: 153,512
Non-trainable params: 0
_________________________________________________________________
input_1 [<keras.engine.node.Node object at 0x0000021F99